In [14]:
import csv
import array, csv, datetime, math, random, re
import numpy as np
import matplotlib.pyplot as plt

def LngLatToWebMercator(lnglat):
    (lng, lat) = lnglat
    x = (lng + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def YearMonthDayToEpoch(year, month, day):
  return (datetime.datetime(int(year), int(month), int(day)) - datetime.datetime(1970, 1, 1)).total_seconds()


In [62]:
FNAME_TMPL = 'TRI_%s_US.csv'
data = []
points = {}
for year in range(1987, 2017):
    fname = FNAME_TMPL % year
    with open(fname) as f:
        reader = csv.DictReader(f)
        for row in reader:
            primaryNaics = row['PRIMARY_NAICS']
            m = re.search("^(3241|3251|3252)",primaryNaics)            
            if not m is None:
                carcinogen = row['CARCINOGEN']
                classification = row['CLASSIFICATION']
                if classification == "DIOXIN" or classification == "DIOXIN AND DIOXIN-LIKE COMPOUNDS":
                    print row
                latitude = row['LATITUDE']
                longitude = row['LONGITUDE']
                totalReleases = row['TOTAL_RELEASES']
                data.append(row)            
                idx =  longitude + 'x' + latitude
                if idx not in points:
                    points[idx] = {
                        year: [{'carcinogen': carcinogen,
                               'classification': classification,
                               'latitude': latitude,
                               'longitude': longitude,
                               'totalReleases': totalReleases
                              }]
                    }
                else:
                    if year not in points[idx]:
                         points[idx][year] = []
                    points[idx][year].append({'carcinogen': carcinogen,
                               'classification': classification,
                               'latitude': latitude,
                               'longitude': longitude,
                               'totalReleases': totalReleases
                              })
                    

In [49]:
len(data)

462255

In [54]:
len(points.keys())

5287

In [55]:
points.keys()[100]

'-96.98922x32.89236'

In [12]:
data[0]

{'': None,
 ' 8.5_RECYCLING_OFF-SITE': '0',
 '5.1_FUGITIVE_AIR': '0',
 '5.2_STACK_AIR': '0',
 '5.3_WATER': '370000',
 '5.4.1_UNDERGROUND_CLASS_I': '0',
 '5.4.2_UNDERGROUND_CLASS_II-V': '0',
 '5.4_UNDERGROUND': '0',
 '5.5.1A_RCRA_C_LANDFILLS': '0',
 '5.5.1B_OTHER_LANDFILLS': '0',
 '5.5.1_LANDFILLS': '1500',
 '5.5.2_LAND_TREATMENT': '0',
 '5.5.3A_RCRA_C_SURFACE_IMP.': '0',
 '5.5.3B_Other_SURFACE_IMP.': '0',
 '5.5.3_SURFACE_IMPOUNDMENT': '0',
 '5.5.4_OTHER_DISPOSAL': '0',
 '6.1_POTW-TOTAL_TRANSFERS': '0',
 '6.1_POTW-TRANSFERS_FOR_RELEASE': '0',
 '6.1_POTW-TRANSFERS_FOR_TREATM.': '0',
 '6.2_M10': '0',
 '6.2_M20': '0',
 '6.2_M24': '0',
 '6.2_M26': '0',
 '6.2_M28': '0',
 '6.2_M40': '0',
 '6.2_M41': '0',
 '6.2_M50': '0',
 '6.2_M54': '0',
 '6.2_M56': '0',
 '6.2_M61': '0',
 '6.2_M62': '0',
 '6.2_M63': '0',
 '6.2_M64': '0',
 '6.2_M65': '0',
 '6.2_M66': '0',
 '6.2_M67': '0',
 '6.2_M69': '0',
 '6.2_M71': '0',
 '6.2_M72': '0',
 '6.2_M73': '0',
 '6.2_M79': '0',
 '6.2_M81': '0',
 '6.2_M82': '0',
 '6.

In [34]:
# 3241 Petroleum and Coal Products Manufacturing
# 3251 Basic Chemical Manufacturing
# 3252 Resin, Synthetic Rubber, and Artificial and Synthetic Fibers and Filaments Manufacturing
# 'PRIMARY_NAICS'
m = re.search("^(3241|3251|3252)",'325110')
m = re.search("^(3241|3251|3252)",'3241000')

In [35]:
m.group()

'3241'

In [60]:
points[points.keys()[4600]]

{1987: [{'carcinogen': 'YES',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '500'}],
 1988: [{'carcinogen': 'YES',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '500'},
  {'carcinogen': 'YES',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '500'}],
 1989: [{'carcinogen': 'NO',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '500'},
  {'carcinogen': 'NO',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '250'}],
 1990: [{'carcinogen': 'NO',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '250'},
  {'carcinogen': 'NO',
   'classification': 'TRI',
   'latitude': '35.06118',
   'longitude': '-85.274261',
   'totalReleases': '500'},
  {'carcinogen': '

In [70]:
#point = x, y, epoch1, epoch_2, PBT_carcinogen, PBT_val_1, PBT_val_2, TRI_carcinogen, TRI_val_1, TRI_val2, Dioxin_carcinogen, Dioxin_val_1, Dioxin_val_2  
totals = {}
for idx in points:
    totals[idx] = {}
    point = points[idx]
    for year in range(1987,2017):
        totals[idx][year] = {}
        PBT_total = 0.0
        TRI_total = 0.0
        PBT_carcinogen = 0.0
        TRI_carcinogen = 0.0
        if year in point:
            for entry in point[year]:
                if entry['classification'] == 'TRI':
                    TRI_total += float(entry['totalReleases']) if entry['totalReleases'] is not None else 0.0
                    if entry['carcinogen'] == 'YES':
                        TRI_carcinogen = 1.0
                if entry['classification'] == 'PBT':
                    PBT_total += float(entry['totalReleases'])
                    if entry['carcinogen'] == 'YES':
                        PBT_carcinogen = 1.0
                        
        totals[idx][year]['PBT_total'] = PBT_total
        totals[idx][year]['PBT_carcinogen'] = PBT_carcinogen
        totals[idx][year]['TRI_total'] = TRI_total
        totals[idx][year]['TRI_carcinogen'] = TRI_carcinogen
        

In [71]:
totals[idx]

{1987: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 0.0},
 1988: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 0.0},
 1989: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 0.0},
 1990: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 0.0},
 1991: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 250.0},
 1992: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 250.0},
 1993: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 1.0,
  'TRI_total': 26265.0},
 1994: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 25255.0},
 1995: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 20.0},
 1996: {'PBT_carcinogen': 0.0,
  'PBT_total': 0.0,
  'TRI_carcinogen': 0.0,
  'TRI_total': 220.0},
 1997: {'PBT_ca

In [76]:
pts = []
for idx in totals:
    lng,lat = idx.split("x")
    for year in range(1987,2015):
        pts += LngLatToWebMercator((float(lng),float(lat))
        pts.append(year)
        pts.append(year+1) 
        pts.append(totals[idx][year]['PBT_carcinogen']) 
        pts.append(totals[idx][year]['PBT_total'])
        pts.append(totals[idx][year+1]['PBT_carcinogen'])
        pts.append(totals[idx][year+1]['PBT_total'])
        pts.append(totals[idx][year][' TRI_carcinogen'])
        pts.append(totals[idx][year]['TRI_total'])
        pts.append(totals[idx][year+1][' TRI_carcinogen'])
        pts.append(totals[idx][year+1]['TRI_total'])
array.array('f', pts).tofile(open('tri-time-series.bin', 'wb'))                                   

SyntaxError: invalid syntax (<ipython-input-76-08bc403feb11>, line 6)

In [74]:
lng

'-93.965034'

In [75]:
lat

'29.825641'

In [78]:
a = []
a.append(1)

In [79]:
a

[1]